---
# Ingenieria de Variables con Pandas  Pivot Table
---

***Tutorial sobre el uso de Pandas Pivot Table***

Desafío:
https://www.kaggle.com/felixzhao/productdemandforecasting


Importamos Librerias

In [1]:
# importamos librerias
import pandas as pd
import numpy as np
import plotly.express as px
pd.set_option('display.max_rows', 350)
pd.set_option('display.max_columns', 350)
pd.set_option('display.width', 1200)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# dataset
df = pd.read_csv(r'Dataset/Historical Product Demand.csv')

# eliminamos nan
df.dropna(inplace=True)

# expresamos correctamente los valores negativos
df['Order_Demand'] = df['Order_Demand'].replace( '[\$,)]','', regex=True ).replace( '[(]','-',   regex=True ).astype(float)

# creamos una columna con el dia del mes, el dia de la semana, el mes, el trimestre, el año y el periodo
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df['Dayofweek'] = df['Date'].dt.dayofweek
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Period'] = df['Date'].dt.to_period('M')
df.sort_values('Date', inplace=True)

# agregamos un 5% de valores NaN en la demanda, y para todas las filas del perido mayo 2012
df.loc[df.sample(int(df.shape[0] * 0.05)).index, ['Order_Demand']] = np.nan
df.loc[(df['Period']=='2012-05'), 'Order_Demand'] = np.nan

# agregamos una feature ficticia que muestra un usuario al azar por cada pedido realizado
df['User'] = np.random.choice(['user 1', 'user 2'], size=df.shape[0])

In [2]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.set_option('display.max_rows', 350)
pd.set_option('display.max_columns', 350)
pd.set_option('display.width', 1200)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

Dataset

In [3]:
df = pd.read_csv(r'Dataset/Historical Product Demand.csv')

In [4]:
df.shape

(1048575, 5)

In [5]:
df.sample(7)

,Product_Code,Warehouse,Product_Category,Date,Order_Demand
868182,Product_1431,Whse_C,Category_019,2016/4/28,20000
974903,Product_1480,Whse_J,Category_019,2016/5/10,1000
563155,Product_1822,Whse_A,Category_018,2014/8/27,5
864804,Product_0033,Whse_S,Category_005,2016/2/3,5000
763322,Product_1111,Whse_J,Category_007,2015/3/25,1
224474,Product_1906,Whse_J,Category_019,2013/1/21,20000
589930,Product_0670,Whse_J,Category_011,2014/9/22,3


Tenemos filas que no poseen una fecha

In [6]:
df.isna().sum()

Product_Code            0
Warehouse               0
Product_Category        0
Date                11239
Order_Demand            0
dtype: int64

Las eliminamos

In [7]:
df.dropna(inplace=True)

In [8]:
df.shape

(1037336, 5)

Examinamos la columna Order_Demand, que hace referencia a la cantidad demandada de un producto

In [9]:
df['Order_Demand'].describe()

count     1037336
unique       3749
top         1000 
freq       112263
Name: Order_Demand, dtype: object

In [10]:
df['Order_Demand'].dtype

dtype('O')

Vemos que existen valores que no permiten utilizar esta columna como numerica, por existir numeros negativos con parentesis, por ejemplo (5). Pasamos estos valores a negativo

In [11]:
df['Order_Demand'] = df['Order_Demand'].replace( '[\$,)]','', regex=True ).replace( '[(]','-',   regex=True ).astype(float)

In [12]:
df['Order_Demand'].describe()

count   1037336.000
mean       4920.153
std       29078.401
min     -999000.000
25%          19.000
50%         300.000
75%        2000.000
max     4000000.000
Name: Order_Demand, dtype: float64

Creamos una columna con el dia del mes, el dia de la semana, el mes, el trimestre, el año y el periodo

In [13]:
df['Date'] = pd.to_datetime(df['Date'], format='%Y/%m/%d')
df['Dayofweek'] = df['Date'].dt.dayofweek
df['Month'] = df['Date'].dt.month
df['Year'] = df['Date'].dt.year
df['Period'] = df['Date'].dt.to_period('M')
df.sort_values('Date', inplace=True)

Agregamos un 5% de valores NaN en la demanda, para algunas filas seleccionadas al azar, y para todas las filas del perido mayo 2012

In [14]:
df.loc[df.sample(int(df.shape[0] * 0.05)).index, ['Order_Demand']] = np.nan

In [15]:
df.loc[(df['Period']=='2012-05'), 'Order_Demand'] = np.nan

In [16]:
df.isna().sum()

Product_Code            0
Warehouse               0
Product_Category        0
Date                    0
Order_Demand        68042
Dayofweek               0
Month                   0
Year                    0
Period                  0
dtype: int64

Agregamos una feature ficticia que muestra un usuario al azar por cada pedido realizado (solo a los fines de ser utilizada en los ejemplos)

In [17]:
df['User'] = np.random.choice(['user 1', 'user 2'], size=df.shape[0])

### Pivot Table

Dataset

In [18]:
df.sample(7)

,Product_Code,Warehouse,Product_Category,Date,Order_Demand,Dayofweek,Month,Year,Period,User
540374,Product_0018,Whse_J,Category_005,2014-09-10,1000.000,2,9,2014,2014-09,user 1
578783,Product_2039,Whse_J,Category_011,2014-06-27,5.000,4,6,2014,2014-06,user 2
335921,Product_1416,Whse_A,Category_019,2013-10-18,0.000,4,10,2013,2013-10,user 1
244948,Product_1410,Whse_A,Category_019,2013-07-24,10000.000,2,7,2013,2013-07,user 1
824850,Product_0031,Whse_J,Category_005,2015-11-24,3000.000,1,11,2015,2015-11,user 1
712706,Product_1388,Whse_J,Category_019,2015-09-09,500.000,2,9,2015,2015-09,user 2
923863,Product_1295,Whse_J,Category_019,2016-08-22,30000.000,0,8,2016,2016-08,user 1


Observamos la cantidad de registros unicos que existen para cada año

In [19]:
df.groupby(['Year']).size()

Year
2011       640
2012    203635
2013    218298
2014    216404
2015    209661
2016    188645
2017        53
dtype: int64

Sin embargo, tambien podriamos utilizar Tablas Pivot, con el indice igual al año, y la funcion 'size'

In [20]:
pd.pivot_table(df,
               index=['Year'],
               aggfunc=['size']
              )

,size
Year,
2011,640
2012,203635
2013,218298
2014,216404
2015,209661
2016,188645
2017,53


Es decir, la tabla pivot puede asemejarse a realizar un grupo, lo que necesitamos es tener un objeto DataFrame, un indice sobre el cual realizaremos la agrupacion de los valores y la funcion a aplicar sobre ellos. En este caso utilizamos el dataset anterior, con el indice igual al año, y la funcion de tamaño *size*.

Pero si observamos, no le brindamos una columna como parametro, por lo que el resultado es un dataframe de una sola columna y se asemeja bastante a un groupby, cuando el mayor poder de las tablas pivot es justamente utilizarlas justamente como tablas, es de decir con parametros de indice (filas) y columnas.

Por ejemplo, si quisieramos saber la **cantidad de registros** que existen **por mes** de cada uno los **años**, podemos hacer un grupo por **Año** como primer nivel, y por **Mes** como segundo nivel

In [21]:
df.groupby(['Year', 'Month']).size().to_frame()[:15]

0
Year Month       
2011 1          1
     5          1
     6          2
     9          4
     10         3
     11        31
     12       598
2012 1      15614
     2      18123
     3      18604
     4      16590
     5      17014
     6      16911
     7      17718
     8      16492

O, tambien podriamos hacer una **tabla pivot**, usando como indice el **Año**, y como columna los **Meses**, donde la visualizacion de los datos es mejor y a su vez las posibilidades de calculo son mayores

In [22]:
pd.pivot_table(df,
               index=['Year'],
               columns=['Month'],
               aggfunc=['size']
              )

size                                                                                                              
Month        1         2         3         4         5         6         7         8         9         10        11        12
Year                                                                                                                         
2011      1.000       nan       nan       nan     1.000     2.000       nan       nan     4.000     3.000    31.000   598.000
2012  15614.000 18123.000 18604.000 16590.000 17014.000 16911.000 17718.000 16492.000 15613.000 18515.000 17416.000 15025.000
2013  16638.000 17119.000 17397.000 17685.000 17786.000 16421.000 19085.000 16660.000 18946.000 24546.000 18617.000 17398.000
2014  18013.000 18214.000 19839.000 18077.000 16639.000 17762.000 18867.000 16021.000 18970.000 19579.000 17486.000 16937.000
2015  18245.000 18116.000 19122.000 17603.000 15572.000 18413.000 19127.000 15507.000 16656.000 17785.000 17335.000 16180.000
2016  14515.000 16130.000 17282.000 15223.000 14487.000 16418.000 15319.000 15333.000 15376.000 16191.000 17335.000 15036.000
2017     53.000       nan       nan       nan       nan       nan       nan       nan       nan       nan       nan       nan

Vemos el indice de la tabla y sus columnas 

In [23]:
print(pd.pivot_table(df,
               index=['Year'],
               columns=['Month'],
               aggfunc=['size']
              ).index)
print(pd.pivot_table(df,
               index=['Year'],
               columns=['Month'],
               aggfunc='size'
              ).columns)

Int64Index([2011, 2012, 2013, 2014, 2015, 2016, 2017], dtype='int64', name='Year')
Int64Index([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12], dtype='int64', name='Month')


si ahora examinamos el indice del **groupby** obsevamos la diferencia con los indices anteriores

In [24]:
df.groupby(['Year', 'Month']).size().index[:15]

MultiIndex([(2011,  1),
            (2011,  5),
            (2011,  6),
            (2011,  9),
            (2011, 10),
            (2011, 11),
            (2011, 12),
            (2012,  1),
            (2012,  2),
            (2012,  3),
            (2012,  4),
            (2012,  5),
            (2012,  6),
            (2012,  7),
            (2012,  8)],
           names=['Year', 'Month'])

y claramente las columnas no existen, ya que es una serie

In [70]:
# df.groupby(['Year', 'Month']).size().columns

El resultado de las tablas pivot es **un objeto del tipo DataFrame**

In [26]:
type(pd.pivot_table(df,
               index=['Year'],
               columns=['Month'],
               aggfunc='size'
              ))

pandas.core.frame.DataFrame

Se puede observar que los registros del año 2011 y 2017 no son representativos, los eliminamos

In [27]:
df = df[~df['Year'].isin([2011,2017])]

Ademas, dentro de las tablas pivot se pueden crear **subniveles**, por ejemplo hacer un indice doble.


Entocnes, agregamos un indice multiple, utilizando como primer nivel codigo del producto, y segundo nivel el Año. Ya que lo que importa es pronosticar la demanda para cada producto

In [28]:
pd.pivot_table(df,
               index=['Product_Code', 'Year'],
               aggfunc=['size']
              )[:10]

size
Product_Code Year      
Product_0001 2012   139
             2013   117
             2014   129
             2015   107
             2016   103
Product_0002 2012    83
             2013    97
             2014   127
             2015    68
             2016    70

Si quisieramos, podriamos complejizar mas el indice, agregando en un tercer nivel que en este caso podría ser el mes

In [29]:
pd.pivot_table(df,
               index=['Product_Code', 'Year', 'Month'],
               aggfunc=['size']
              )[:30]

size
Product_Code Year Month      
Product_0001 2012 1        15
                  2        13
                  3        13
                  4         7
                  5        10
                  6         7
                  7        10
                  8        14
                  9         9
                  10       20
                  11       11
                  12       10
             2013 1         7
                  2         9
                  3        11
                  4        10
                  5         9
                  6        10
                  7        12
                  8         5
                  9         9
                  10       12
                  11       12
                  12       11
             2014 1         9
                  2        10
                  3        10
                  4         8
                  5         7
                  6        11

Sin embargo, es mas sencillo trabajar con el **periodo** que utilizar como segundo nivel el año y tercer nivel el mes

In [30]:
pd.pivot_table(df,
               index=['Product_Code', 'Period'],
               aggfunc=['size']
              )[:10]

size
Product_Code Period       
Product_0001 2012-01    15
             2012-02    13
             2012-03    13
             2012-04     7
             2012-05    10
             2012-06     7
             2012-07    10
             2012-08    14
             2012-09     9
             2012-10    20

Ahora lo que vamos hacer es agregar columnas a nuestra tabla, ya que se necesita predecir la cantidad demandada para cada mes pero tambien para cada almacén. Por lo tanto agregamos como columna el tipo de almacén

In [31]:
pd.pivot_table(df,
               index=['Product_Code', 'Period'],
               columns=['Warehouse'],
               aggfunc=['size']
              )[:10]

size                     
Warehouse            Whse_A Whse_C Whse_J Whse_S
Product_Code Period                             
Product_0001 2012-01  7.000    nan  8.000    nan
             2012-02  5.000    nan  8.000    nan
             2012-03  6.000    nan  7.000    nan
             2012-04  2.000    nan  5.000    nan
             2012-05  4.000    nan  6.000    nan
             2012-06  1.000    nan  6.000    nan
             2012-07  3.000    nan  7.000    nan
             2012-08  8.000    nan  6.000    nan
             2012-09  5.000    nan  4.000    nan
             2012-10  8.000    nan 12.000    nan

Existen valores NaN para algunas Almacenes, esto signfica que para ese producto en ese periodo y para esa almacén no existe un registro, por ejemplo en el producto 0002 existen valores NaN en otras almacenes distintas a 'Whse_C' y 'Whse_S', cuando para del producto 0001 sucede lo contrario

In [32]:
pd.pivot_table(df,
               index=['Product_Code', 'Period'],
               columns=['Warehouse'],
               aggfunc=['size']
              )[60:70]

size                     
Warehouse            Whse_A Whse_C Whse_J Whse_S
Product_Code Period                             
Product_0002 2012-01    nan    nan    nan  3.000
             2012-02    nan  2.000    nan  1.000
             2012-03    nan    nan    nan  4.000
             2012-04    nan  1.000    nan  4.000
             2012-05    nan    nan    nan  9.000
             2012-06    nan  1.000    nan  9.000
             2012-07    nan  1.000    nan  8.000
             2012-08    nan  1.000    nan  7.000
             2012-09    nan  1.000    nan  9.000
             2012-10    nan    nan    nan  6.000

Completamos los valores NaN con 0 ya que significa que ese producto no tuvo un registro,
utilizando **fill_value = 0**

In [33]:
pd.pivot_table(df,
               index=['Product_Code', 'Period'],
               columns=['Warehouse'],
               aggfunc=['size'],
               fill_value=0
              )

size                     
Warehouse            Whse_A Whse_C Whse_J Whse_S
Product_Code Period                             
Product_0001 2012-01      7      0      8      0
             2012-02      5      0      8      0
             2012-03      6      0      7      0
             2012-04      2      0      5      0
             2012-05      4      0      6      0
...                     ...    ...    ...    ...
Product_2172 2016-07      0      0      3      0
             2016-08      0      0      2      0
             2016-09      0      0      3      0
             2016-10      0      0      2      0
             2016-11      0      0      2      0

[105378 rows x 4 columns]

Hasta ahora tenemos solo indices, columnas, y funciones, y siempre hemos calculado el tamaño total de los registros para cada indice (**funcion size**) la cual se aplica sobre ***todas las columnas del dataset***, pero como la funcion size unicamente cuenta la cantidad de filas, el valor es el **mismo** para todas las columnas y no existe diferencias entre ellas, en cambio si cambiamos la funcion **size** en **aggfunc** por otra funcion como **count** podemos ver que ahora el calculo de la funcion ser realiza para **cada columna del dataset**  pero **sin incluir las columnas definidas dentro de los parametros como indices o como columnas del la tabla pivot**.

In [34]:
pd.pivot_table(df,
               index=['Product_Code', 'Period'],
               columns=['Warehouse'],
               aggfunc=['count'],
               fill_value=0
              )[:10]

count                                                                                                                                                                                                                
                       Date                      Dayofweek                       Month                      Order_Demand                      Product_Category                        User                        Year                     
Warehouse            Whse_A Whse_C Whse_J Whse_S    Whse_A Whse_C Whse_J Whse_S Whse_A Whse_C Whse_J Whse_S       Whse_A Whse_C Whse_J Whse_S           Whse_A Whse_C Whse_J Whse_S Whse_A Whse_C Whse_J Whse_S Whse_A Whse_C Whse_J Whse_S
Product_Code Period                                                                                                                                                                                                                        
Product_0001 2012-01      7      0      8      0         7      0      8      0      7      0      8      0            6      0      6      0                7      0      8      0      7      0      8      0      7      0      8      0
             2012-02      5      0      8      0         5      0      8      0      5      0      8      0            5      0      8      0                5      0      8      0      5      0      8      0      5      0      8      0
             2012-03      6      0      7      0         6      0      7      0      6      0      7      0            6      0      6      0                6      0      7      0      6      0      7      0      6      0      7      0
             2012-04      2      0      5      0         2      0      5      0      2      0      5      0            2      0      5      0                2      0      5      0      2      0      5      0      2      0      5      0
             2012-05      4      0      6      0         4      0      6      0      4      0      6      0            0      0      0      0                4      0      6      0      4      0      6      0      4      0      6      0
             2012-06      1      0      6      0         1      0      6      0      1      0      6      0            0      0      6      0                1      0      6      0      1      0      6      0      1      0      6      0
             2012-07      3      0      7      0         3      0      7      0      3      0      7      0            3      0      7      0                3      0      7      0      3      0      7      0      3      0      7      0
             2012-08      8      0      6      0         8      0      6      0      8      0      6      0            7      0      6      0                8      0      6      0      8      0      6      0      8      0      6      0
             2012-09      5      0      4      0         5      0      4      0      5      0      4      0            5      0      3      0                5      0      4      0      5      0      4      0      5      0      4      0
             2012-10      8      0     12      0         8      0     12      0      8      0     12      0            8      0     11      0                8      0     12      0      8      0     12      0      8      0     12      0

Entonces, si queremos aplicar una funcion sobre una **determinada feature o columna** del dataset debemos agregar un parametro **values** que se refiere a la columna del dataset original sobre la cual vamos a aplicar la funciones defindas en **aggfunc**, sino la funcion se aplicara sobre todas las features salvo aquellas que esten definidas como parametros en los idices o columnas. 

Supongamos que solamente queremos contar la cantidad de valores sobre las columnas 'Order_Demand' y la columna 'Dayofweek', entonces pasamos como lista a estas nombre de columnas el el parametro values

In [35]:
pd.pivot_table(df,
               index=['Product_Code', 'Period'],
               columns=['Warehouse'],
               values=['Order_Demand', 'Dayofweek'],
               aggfunc=['count'],
               fill_value=0
              )[:10]

count                                                       
                     Dayofweek                      Order_Demand                     
Warehouse               Whse_A Whse_C Whse_J Whse_S       Whse_A Whse_C Whse_J Whse_S
Product_Code Period                                                                  
Product_0001 2012-01         7      0      8      0            6      0      6      0
             2012-02         5      0      8      0            5      0      8      0
             2012-03         6      0      7      0            6      0      6      0
             2012-04         2      0      5      0            2      0      5      0
             2012-05         4      0      6      0            0      0      0      0
             2012-06         1      0      6      0            0      0      6      0
             2012-07         3      0      7      0            3      0      7      0
             2012-08         8      0      6      0            7      0      6      0
             2012-09         5      0      4      0            5      0      3      0
             2012-10         8      0     12      0            8      0     11      0

Si quisieramos podriamos agregar mas columnas, por ejemplo podriamos agregar el usuario que realiza el pedido, para esto agregamos una lista con el nombre de las columnas, y el orden de esta lista marcará el orden de los distintos sub niveles. En este caso se realiza una division de tipo de usuario por cada almacen existente

In [36]:
pd.pivot_table(df,
               index=['Product_Code', 'Period'],
               columns=['Warehouse', 'User'],
               values=['Order_Demand', 'Dayofweek'],
               aggfunc=['count'],
               fill_value=0
              )[:10]

count                                                                                                               
                     Dayofweek                                                  Order_Demand                                                 
Warehouse               Whse_A        Whse_C        Whse_J        Whse_S              Whse_A        Whse_C        Whse_J        Whse_S       
User                    user 1 user 2 user 1 user 2 user 1 user 2 user 1 user 2       user 1 user 2 user 1 user 2 user 1 user 2 user 1 user 2
Product_Code Period                                                                                                                          
Product_0001 2012-01         3      4      0      0      2      6      0      0            3      3      0      0      2      4      0      0
             2012-02         2      3      0      0      7      1      0      0            2      3      0      0      7      1      0      0
             2012-03         2      4      0      0      3      4      0      0            2      4      0      0      2      4      0      0
             2012-04         1      1      0      0      2      3      0      0            1      1      0      0      2      3      0      0
             2012-05         1      3      0      0      2      4      0      0            0      0      0      0      0      0      0      0
             2012-06         0      1      0      0      1      5      0      0            0      0      0      0      1      5      0      0
             2012-07         1      2      0      0      3      4      0      0            1      2      0      0      3      4      0      0
             2012-08         4      4      0      0      3      3      0      0            3      4      0      0      3      3      0      0
             2012-09         3      2      0      0      2      2      0      0            3      2      0      0      1      2      0      0
             2012-10         4      4      0      0      4      8      0      0            4      4      0      0      3      8      0      0

Entonces lo que estamos viendo es: cantidad de registros de demanda para cada producto, por cada periodo, de acuerdo a el almacen que realizo el pedido y a el usuario que la realizó.

Esto se *asemeja* a tener dos **groupby**, uno de acuerdo a los indices ['Product_Code', 'Period'], y otro a las columnas ['Warehouse', 'User'], presentados ambos en un tabla donde se realiza el producto matricial de ambos para los valores de las features ['Order_Demand', 'Dayofweek']

In [37]:
df.groupby(['Product_Code', 'Period'])['Order_Demand', 'Dayofweek'].count()

Order_Demand  Dayofweek
Product_Code Period                          
Product_0001 2012-01            12         15
             2012-02            13         13
             2012-03            12         13
             2012-04             7          7
             2012-05             0         10
...                            ...        ...
Product_2172 2016-07             2          3
             2016-08             1          2
             2016-09             3          3
             2016-10             2          2
             2016-11             2          2

[105378 rows x 2 columns]

In [38]:
df.groupby(['Warehouse', 'User'])['Order_Demand', 'Dayofweek'].count().T

Warehouse    Whse_A        Whse_C         Whse_J         Whse_S       
User         user 1 user 2 user 1 user 2  user 1  user 2 user 1 user 2
Order_Demand  65730  66678  19737  19779  357835  356960  40909  41009
Dayofweek     70588  71609  21127  21080  382484  381731  43949  44075

Pero ahora volvamos a la tabla anterior, sin tener en cuenta los usuarios ya que esta es una variable ficticia del dataset a solo efecto de mostrar las posibilidades de las tablas pivot

In [39]:
pd.pivot_table(df,
               index=['Product_Code', 'Period'],
               columns=['Warehouse'],
               values=['Order_Demand', 'Dayofweek'],
               aggfunc=['count'],
               fill_value=0
              )[:10]

count                                                       
                     Dayofweek                      Order_Demand                     
Warehouse               Whse_A Whse_C Whse_J Whse_S       Whse_A Whse_C Whse_J Whse_S
Product_Code Period                                                                  
Product_0001 2012-01         7      0      8      0            6      0      6      0
             2012-02         5      0      8      0            5      0      8      0
             2012-03         6      0      7      0            6      0      6      0
             2012-04         2      0      5      0            2      0      5      0
             2012-05         4      0      6      0            0      0      0      0
             2012-06         1      0      6      0            0      0      6      0
             2012-07         3      0      7      0            3      0      7      0
             2012-08         8      0      6      0            7      0      6      0
             2012-09         5      0      4      0            5      0      3      0
             2012-10         8      0     12      0            8      0     11      0

Debemos realizar una pequeña aclaracion acerca del funcionamiento de la función **size**, ya que esta cuenta el total de registros (o filas) *sin importar los valores NaN* que existen dentro de la columna sobre la cual aplicamos la función.
La cual es distinta de la funcion **count** que unicamente cuenta los registros no nulos.

Para poder comparar esto, podemos hacer el uso *listas de funciones* dentro del parametro **aggfunc**

In [40]:
pd.pivot_table(df,
               index=['Product_Code', 'Period'],
               values=['Order_Demand'],
               aggfunc=['count', 'size'],
               fill_value=0
              )[:10]

count size
                     Order_Demand    0
Product_Code Period                   
Product_0001 2012-01           12   15
             2012-02           13   13
             2012-03           12   13
             2012-04            7    7
             2012-05            0   10
             2012-06            6    7
             2012-07           10   10
             2012-08           13   14
             2012-09            8    9
             2012-10           19   20

Sin embargo, si quisieramos aplicar ambas funciones pero introducciendo el parametro **columns=['Warehouse']** como se realizó anteriormente, obtendriamos un error, ya que los indices de ambas funciones no van a ser los mismos, y esto viene dado porque cada funcion se aplica sobre **todas las columnas del dataset**, como se explico más arriba. 

In [72]:
# pd.pivot_table(df,
#                index=['Product_Code', 'Period'],
#                columns=['Warehouse'],
#                values=['Order_Demand'],
#                aggfunc=['size',  'count'],
#                fill_value=0
#               )[:10]

Para solucionar esto, debemos definir **qué tipo de funcion** aplicamos a cada columna del dataset, y a esto lo hacemos pasandole un diccionario en el parametro **aggfunc** con el nombre de la columna / valor / indice del dataset, y la funcion a aplicar.
En este caso le pasamos una *lista de funciones* ('count', 'size')

In [42]:
pd.pivot_table(data=df,
               index=['Product_Code', 'Period'],
               columns=['Warehouse'],
               values=['Order_Demand'],
               aggfunc={'Order_Demand':['count', 'size']},
               fill_value=0
              )[0:10]

Order_Demand                                                 
                            count                        size                     
Warehouse                  Whse_A Whse_C Whse_J Whse_S Whse_A Whse_C Whse_J Whse_S
Product_Code Period                                                               
Product_0001 2012-01            6      0      6      0      7      0      8      0
             2012-02            5      0      8      0      5      0      8      0
             2012-03            6      0      6      0      6      0      7      0
             2012-04            2      0      5      0      2      0      5      0
             2012-05            0      0      0      0      4      0      6      0
             2012-06            0      0      6      0      1      0      6      0
             2012-07            3      0      7      0      3      0      7      0
             2012-08            7      0      6      0      8      0      6      0
             2012-09            5      0      3      0      5      0      4      0
             2012-10            8      0     11      0      8      0     12      0

Podemos aplicar **diferentes fuciones** a diferentes **valores**, por ejemplo podemos **contar** la cantidad de pedidos y la **cantidad de pedidos por dia del mes** en que se realizaron los pedidos (**nunique**), para entender si los pedidos se realizan siempre el mismo dia

In [43]:
pd.pivot_table(data=df,
               index=['Product_Code', 'Period'],
               columns=['Warehouse'],
               values=['Date', 'Order_Demand'],
               aggfunc={'Date': 'nunique', 'Order_Demand':'count'},
               fill_value=0,
              )[:10]

Date                      Order_Demand                     
Warehouse            Whse_A Whse_C Whse_J Whse_S       Whse_A Whse_C Whse_J Whse_S
Product_Code Period                                                               
Product_0001 2012-01      5      0      7      0            6      0      6      0
             2012-02      5      0      7      0            5      0      8      0
             2012-03      5      0      5      0            6      0      6      0
             2012-04      2      0      5      0            2      0      5      0
             2012-05      3      0      5      0            0      0      0      0
             2012-06      1      0      5      0            0      0      6      0
             2012-07      3      0      7      0            3      0      7      0
             2012-08      7      0      5      0            7      0      6      0
             2012-09      4      0      4      0            5      0      3      0
             2012-10      6      0      9      0            8      0     11      0

Tambien podriamos aplicar las funciones sobre algunas de las features elegidas como **columnas** en el tabla pivot (*columns*), por ejemplo en este caso *Warehouse*, y al mismo tiempo otra funcion sobre los valores definidos en *values*

In [44]:
pd.pivot_table(data=df,
               index=['Product_Code', 'Period'],
               columns=['Warehouse'],
               values=['Date'],
               aggfunc={'Date': 'nunique', 'Warehouse':'count'},
               fill_value=0,
              )[:10]

Date                      Warehouse                     
Warehouse            Whse_A Whse_C Whse_J Whse_S    Whse_A Whse_C Whse_J Whse_S
Product_Code Period                                                            
Product_0001 2012-01      5      0      7      0         7      0      8      0
             2012-02      5      0      7      0         5      0      8      0
             2012-03      5      0      5      0         6      0      7      0
             2012-04      2      0      5      0         2      0      5      0
             2012-05      3      0      5      0         4      0      6      0
             2012-06      1      0      5      0         1      0      6      0
             2012-07      3      0      7      0         3      0      7      0
             2012-08      7      0      5      0         8      0      6      0
             2012-09      4      0      4      0         5      0      4      0
             2012-10      6      0      9      0         8      0     12      0

O tambien funciones sobre uno de los **indices**

In [45]:
pd.pivot_table(data=df,
               index=['Product_Code', 'Period'],
               columns=['Warehouse'],
               values=['Date'],
               aggfunc={'Date': 'nunique', 'Product_Code':'count'},
               fill_value=0,
              )[:10]

Date                      Product_Code                     
Warehouse            Whse_A Whse_C Whse_J Whse_S       Whse_A Whse_C Whse_J Whse_S
Product_Code Period                                                               
Product_0001 2012-01      5      0      7      0            7      0      8      0
             2012-02      5      0      7      0            5      0      8      0
             2012-03      5      0      5      0            6      0      7      0
             2012-04      2      0      5      0            2      0      5      0
             2012-05      3      0      5      0            4      0      6      0
             2012-06      1      0      5      0            1      0      6      0
             2012-07      3      0      7      0            3      0      7      0
             2012-08      7      0      5      0            8      0      6      0
             2012-09      4      0      4      0            5      0      4      0
             2012-10      6      0      9      0            8      0     12      0

Tambien podriamos obtener un **subtotal** para cada campo calculado con **margins=True**

In [46]:
pd.pivot_table(data=df,
               index=['Product_Code', 'Period'],
               columns=['Warehouse'],
               values=['Order_Demand'],
               aggfunc=['count'],
               fill_value=0,
               margins=True,
              )

count                                 
                     Order_Demand                                 
Warehouse                  Whse_A Whse_C  Whse_J Whse_S        All
Product_Code Period                                               
Product_0001 2012-01            6      0       6      0     12.000
             2012-02            5      0       8      0     13.000
             2012-03            6      0       6      0     12.000
             2012-04            2      0       5      0      7.000
             2012-05            0      0       0      0        nan
...                           ...    ...     ...    ...        ...
Product_2172 2016-08            0      0       1      0      1.000
             2016-09            0      0       3      0      3.000
             2016-10            0      0       2      0      2.000
             2016-11            0      0       2      0      2.000
All                        132408  39516  714795  81918 968637.000

[105379 rows x 5 columns]

Sin embargo los subtotales no son posibles cuando utilzamos la funcion 'size' ya que arroja un *KeyError*

In [74]:
# pd.pivot_table(data=df,
#                index=['Product_Code', 'Period'],
#                columns=['Warehouse'],
#                values=['Order_Demand'],
#                aggfunc={'Order_Demand':'size'},
#                fill_value=0,
#                margins=True
#               )[0:10]

Dentro de las funciones que aplicamos podemos crear funciones **lambda**. Por ejemplo, en este caso contamos las cantidades de demanda que superan las 100000 unidades demandadas

In [48]:
pd.pivot_table(data=df,
               index=['Product_Code', 'Period'],
               columns=['Warehouse'],
               values=['Order_Demand'],
               aggfunc={'Order_Demand':lambda x: (x > 100000).count()},
              )[0:10]

Order_Demand                     
Warehouse                  Whse_A Whse_C Whse_J Whse_S
Product_Code Period                                   
Product_0001 2012-01        7.000    nan  8.000    nan
             2012-02        5.000    nan  8.000    nan
             2012-03        6.000    nan  7.000    nan
             2012-04        2.000    nan  5.000    nan
             2012-05        4.000    nan  6.000    nan
             2012-06        1.000    nan  6.000    nan
             2012-07        3.000    nan  7.000    nan
             2012-08        8.000    nan  6.000    nan
             2012-09        5.000    nan  4.000    nan
             2012-10        8.000    nan 12.000    nan

O podemos utilizar funciones de Pandas o Numpy

In [49]:
pd.pivot_table(data=df,
               index=['Product_Code', 'Period'],
               columns=['Warehouse'],
               values=['Order_Demand'],
               aggfunc={'Order_Demand': [pd.Series.nunique, np.sum]},
              )[0:10]

Order_Demand                                                      
                          nunique                            sum                       
Warehouse                  Whse_A Whse_C Whse_J Whse_S    Whse_A Whse_C   Whse_J Whse_S
Product_Code Period                                                                    
Product_0001 2012-01        4.000    nan  4.000    nan  6200.000    nan 3100.000    nan
             2012-02        3.000    nan  5.000    nan 10400.000    nan 2600.000    nan
             2012-03        4.000    nan  3.000    nan 11000.000    nan 2100.000    nan
             2012-04        2.000    nan  3.000    nan  1500.000    nan 1800.000    nan
             2012-05        0.000    nan  0.000    nan     0.000    nan    0.000    nan
             2012-06        0.000    nan  5.000    nan     0.000    nan 4400.000    nan
             2012-07        3.000    nan  4.000    nan  8000.000    nan 2100.000    nan
             2012-08        6.000    nan  6.000    nan  6200.000    nan 2900.000    nan
             2012-09        4.000    nan  2.000    nan  5600.000    nan 1000.000    nan
             2012-10        7.000    nan  5.000    nan 11700.000    nan 3200.000    nan

Acceder a los elementos de la tabla es de la misma forma que para acceder a los elementos de un DataFrame (ya que el resultado de Pivot Table es un DataFrame, como se menciono anteriormente).

Por ejemplo, podemos hacer una division entre la cantidad demandada por cada almacen central, y el total demandado en el mes, para obtener el porcentaje de participacion de cada uno de los almacenes en la demanda del mes por producto.


Para eso, primero creamos una tabla que tiene la cantidad total demandada por cada almacen, haciendo uso de la  funcion **sum**, y agregamos el total demandado (margins=True)

In [50]:
pivot_table = pd.pivot_table(data=df,
                                index=['Product_Code', 'Period'],
                                columns=['Warehouse'],
                                values=['Order_Demand'],
                                aggfunc={'Order_Demand': 'sum'},
                                fill_value=0,
                                margins=True
                            )

In [51]:
pivot_table[:15]

Order_Demand                               
Warehouse                  Whse_A Whse_C Whse_J Whse_S       All
Product_Code Period                                             
Product_0001 2012-01         6200      0   3100      0  9300.000
             2012-02        10400      0   2600      0 13000.000
             2012-03        11000      0   2100      0 13100.000
             2012-04         1500      0   1800      0  3300.000
             2012-05            0      0      0      0       nan
             2012-06            0      0   4400      0  4400.000
             2012-07         8000      0   2100      0 10100.000
             2012-08         6200      0   2900      0  9100.000
             2012-09         5600      0   1000      0  6600.000
             2012-10        11700      0   3200      0 14900.000
             2012-11         4500      0   2400      0  6900.000
             2012-12         5300      0    700      0  6000.000
             2013-01         5000      0   1900      0  6900.000
             2013-02         7700      0    600      0  8300.000
             2013-03        10600      0   1500      0 12100.000

Luego utilizamos la funcion **divide** de pandas para hacer una division de *cada columna* por el *total*.

Para acceder a cada elemento hacemos uso de la funcion **loc** o tambien de **tuplas** con los nombres de las columnas

In [52]:
pivot_table_percentage = pivot_table['Order_Demand'].loc[:,['Whse_A', 'Whse_C', 'Whse_J', 'Whse_S']]\
                                .divide(pivot_table[('Order_Demand', 'All')], 
                                        axis=0).fillna(0)
pivot_table_percentage

Warehouse             Whse_A  Whse_C  Whse_J  Whse_S
Product_Code Period                                 
Product_0001 2012-01   0.667   0.000   0.333   0.000
             2012-02   0.800   0.000   0.200   0.000
             2012-03   0.840   0.000   0.160   0.000
             2012-04   0.455   0.000   0.545   0.000
             2012-05   0.000   0.000   0.000   0.000
...                      ...     ...     ...     ...
Product_2172 2016-08   0.000   0.000   1.000   0.000
             2016-09   0.000   0.000   1.000   0.000
             2016-10   0.000   0.000   1.000   0.000
             2016-11   0.000   0.000   1.000   0.000
All                    0.028   0.114   0.657   0.201

[105379 rows x 4 columns]

Ahora, creamos una nueva tabla, que contiene la cantidad total de pedidos de ese producto, la cantidad promedio, y la cantidad de dias unicos que se realizo el pedido.
A esto lo hacemos sin dividir en almacenes (columnas) como se hizo anteriormente, solamente tomando el total por periodo.

In [53]:
pivot_table_total = pd.pivot_table(data=df,
                                index=['Product_Code', 'Period'],
                                values=['Order_Demand', 'Date'],
                                aggfunc={'Date': 'nunique',
                                         'Order_Demand': ['mean', 'sum']},
                                fill_value=0     
                            )
pivot_table_total

Date Order_Demand       
                     nunique         mean    sum
Product_Code Period                             
Product_0001 2012-01      11      775.000   9300
             2012-02      10     1000.000  13000
             2012-03      10     1091.667  13100
             2012-04       7      471.429   3300
             2012-05       7        0.000      0
...                      ...          ...    ...
Product_2172 2016-07       2      200.000    400
             2016-08       2      300.000    300
             2016-09       3      400.000   1200
             2016-10       2      450.000    900
             2016-11       2      450.000    900

[105378 rows x 3 columns]

Y si queremos, a esta ultima tabla de 'totales generales por periodo' la podemos cruzar con la informacion de la tabla por almacen creada anteriormente, haciendo uso de la funcion **merge** y diciendole que utilize como indice los indices utilizados para crear dichas tablas ['Product_Code', 'Period']

In [54]:
pd.merge(pivot_table_percentage, pivot_table_total, on=['Product_Code', 'Period'])

/home/matiasscorsetti/anaconda3/lib/python3.7/site-packages/pandas/core/reshape/merge.py:617: UserWarning:

merging between different levels can give an unintended result (1 levels on the left, 2 on the right)



Whse_A  Whse_C  Whse_J  Whse_S  (Date, nunique)  (Order_Demand, mean)  (Order_Demand, sum)
Product_Code Period                                                                                             
Product_0001 2012-01   0.667   0.000   0.333   0.000               11               775.000                 9300
             2012-02   0.800   0.000   0.200   0.000               10              1000.000                13000
             2012-03   0.840   0.000   0.160   0.000               10              1091.667                13100
             2012-04   0.455   0.000   0.545   0.000                7               471.429                 3300
             2012-05   0.000   0.000   0.000   0.000                7                 0.000                    0
...                      ...     ...     ...     ...              ...                   ...                  ...
Product_2172 2016-07   0.000   0.000   1.000   0.000                2               200.000                  400
             2016-08   0.000   0.000   1.000   0.000                2               300.000                  300
             2016-09   0.000   0.000   1.000   0.000                3               400.000                 1200
             2016-10   0.000   0.000   1.000   0.000                2               450.000                  900
             2016-11   0.000   0.000   1.000   0.000                2               450.000                  900

[105378 rows x 7 columns]

De ser necesario, podemos *aplanar* las columnas para eliminar el **MultiIndex** de las mismas, ya que como vemos nos alerta por el hecho de haber dos niveles (en las columnas) en la tabla derecha y uno solo nivel en la tabla izquierda

In [55]:
pivot_table_total.columns = ['_'.join(col).strip() for col in pivot_table_total.columns.values]

In [56]:
pd.merge(pivot_table_percentage, pivot_table_total, on=['Product_Code', 'Period'])

Whse_A  Whse_C  Whse_J  Whse_S  Date_nunique  Order_Demand_mean  Order_Demand_sum
Product_Code Period                                                                                    
Product_0001 2012-01   0.667   0.000   0.333   0.000            11            775.000              9300
             2012-02   0.800   0.000   0.200   0.000            10           1000.000             13000
             2012-03   0.840   0.000   0.160   0.000            10           1091.667             13100
             2012-04   0.455   0.000   0.545   0.000             7            471.429              3300
             2012-05   0.000   0.000   0.000   0.000             7              0.000                 0
...                      ...     ...     ...     ...           ...                ...               ...
Product_2172 2016-07   0.000   0.000   1.000   0.000             2            200.000               400
             2016-08   0.000   0.000   1.000   0.000             2            300.000               300
             2016-09   0.000   0.000   1.000   0.000             3            400.000              1200
             2016-10   0.000   0.000   1.000   0.000             2            450.000               900
             2016-11   0.000   0.000   1.000   0.000             2            450.000               900

[105378 rows x 7 columns]

### Stack & Unstack

Primero guardamos la union de las tablas anteriores en una nueva variable *merge_pivot_table*

In [57]:
merge_pivot_table = pd.merge(pivot_table_percentage, pivot_table_total, on=['Product_Code', 'Period'])

Pasemos ahora a entender como funciona **stack** y **unstack**

**stack** lo que hace es utilizar a las **columnas** del DataFrame como un **indice**

In [58]:
merge_pivot_table.stack()\
    .to_frame()[:14]

0
Product_Code Period                             
Product_0001 2012-01 Whse_A                0.667
                     Whse_C                0.000
                     Whse_J                0.333
                     Whse_S                0.000
                     Date_nunique         11.000
                     Order_Demand_mean   775.000
                     Order_Demand_sum   9300.000
             2012-02 Whse_A                0.800
                     Whse_C                0.000
                     Whse_J                0.200
                     Whse_S                0.000
                     Date_nunique         10.000
                     Order_Demand_mean  1000.000
                     Order_Demand_sum  13000.000

Por el contrario, **unstack** lo que hace es llevar los **indices** a **columnas**

In [59]:
merge_pivot_table.unstack()

Whse_A                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Whse_C                                                                                                                                                                                                                                                                                                                                                                                                                                                                                          Whse_J                                                                                                                                                                                                                          \
Period       2012-01 2012-02 2012-03 2012-04 2012-05 2012-06 2012-07 2012-08 2012-09 2012-10 2012-11 2012-12 2013-01 2013-02 2013-03 2013-04 2013-05 2013-06 2013-07 2013-08 2013-09 2013-10 2013-11 2013-12 2014-01 2014-02 2014-03 2014-04 2014-05 2014-06 2014-07 2014-08 2014-09 2014-10 2014-11 2014-12 2015-01 2015-02 2015-03 2015-04 2015-05 2015-06 2015-07 2015-08 2015-09 2015-10 2015-11 2015-12 2016-01 2016-02 2016-03 2016-04 2016-05 2016-06 2016-07 2016-08 2016-09 2016-10 2016-11 2016-12 2012-01 2012-02 2012-03 2012-04 2012-05 2012-06 2012-07 2012-08 2012-09 2012-10 2012-11 2012-12 2013-01 2013-02 2013-03 2013-04 2013-05 2013-06 2013-07 2013-08 2013-09 2013-10 2013-11 2013-12 2014-01 2014-02 2014-03 2014-04 2014-05 2014-06 2014-07 2014-08 2014-09 2014-10 2014-11 2014-12 2015-01 2015-02 2015-03 2015-04 2015-05 2015-06 2015-07 2015-08 2015-09 2015-10 2015-11 2015-12 2016-01 2016-02 2016-03 2016-04 2016-05 2016-06 2016-07 2016-08 2016-09 2016-10 2016-11 2016-12 2012-01 2012-02 2012-03 2012-04 2012-05 2012-06 2012-07 2012-08 2012-09 2012-10 2012-11 2012-12 2013-01 2013-02 2013-03 2013-04 2013-05 2013-06 2013-07 2013-08 2013-09 2013-10 2013-11 2013-12 2014-01 2014-02 2014-03 2014-04   
Product_Code                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
Product_0001   0.667   0.800   0.840   0.455   0.000   0.000   0.792   0.681   0.848   0.785   0.652   0.883   0.725   0.928   0.876   0.800   0.737   0.938   0.711   0.682   0.907   0.847   0.778   0.536   0.661   0.460   0.607   0.921   0.810   0.907   0.653   0.812   0.742   0.936   0.710   0.750   0.724   0.816   0.514   0.707   0.907   0.588   0.831   0.921   0.889   0.435   0.556   0.455   0.048   0.825  

Dentro de las opciones de estas funciones, se puede establecer **qué nivel** es el que se desea llevar a indice (stack) o a columna (unstack)

Por ejemplo, ahora llevamos a columna cada uno de los productos (level=0), quedando como indice unicamente el periodo

In [60]:
merge_pivot_table.unstack(level=0)[:10]

Whse_A                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                    \
Product_Code Product_0001 Product_0002 Product_0003 Product_0004 Product_0005 Product_0006 Product_0007 Product_0008 Product_0009 Product_0010 Product_0011 Product_0012 Product_0013 Product_0014 Product_0015 Product_0016 Product_0017 Product_0018 Product_0019 Product_0020 Product_0021 Product_0022 Product_0023 Product_0024 Product_0025 Product_0026 Product_0027 Product_0028 Product_0029 Product_0030 Product_0031 Product_0032 Product_0033 Product_0034 Product_0035 Product_0036 Product_0037 Product_0038 Product_0039 Product_0040 Product_0041 Product_0042 Product_0043 Product_0044 Product_0045 Product_0046 Product_0047 Product_0048 Product_0049 Product_0050 Product_0051 Product_0052 Product_0053 Product_0054 Product_0055 Product_0056 Product_0057 Product_0058 Product_0059 Product_0060 Product_0061 Product_0062 Product_0063 Product_0064 Product_0065 Product_0066 Product_0067 Product_0068 Product_0069 Product_0070 Product_0071 Product_0072 Product_0073 Product_0074 Product_0075 Product_0076 Product_0077 Product_0078 Product_0079 Product_0080 Product_0081 Product_0082 Product_0083 Product_0084 Product_0085 Product_0086 Product_0087 Product_0088 Product_0089 Product_0090 Product_0091   
Period                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
2012-01             0.667        0.000        0.000        0.000          nan          nan        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.154        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.000        0.154       

### Crosstab

El uso practico de **Crosstab** en pandas es muy similar al de **Pivot Tables**, de hecho los resultados obtenidos son los mismos, pero difieren en la manera en que estos objetos se crean.
**Pivot Tables** espera un *DataFrame* como entrada, en cambio **Crosstab** utiliza *Arrays* o *Series* para crear el objeto. Veamos un ejemplo:

Al comienzo del tutorial creamos una tabla pivot que contenía la cantidad de registros utilizando como indice los **años** y como columnas los **meses**

In [61]:
pd.pivot_table(df,
               index=['Year'],
               columns=['Month'],
               aggfunc=['size']
              )

size                                                                             
Month     1      2      3      4      5      6      7      8      9      10     11     12
Year                                                                                     
2012   15614  18123  18604  16590  17014  16911  17718  16492  15613  18515  17416  15025
2013   16638  17119  17397  17685  17786  16421  19085  16660  18946  24546  18617  17398
2014   18013  18214  19839  18077  16639  17762  18867  16021  18970  19579  17486  16937
2015   18245  18116  19122  17603  15572  18413  19127  15507  16656  17785  17335  16180
2016   14515  16130  17282  15223  14487  16418  15319  15333  15376  16191  17335  15036

A esta misma tabla la podemos hacer con **CrossTab**, a partir de una lista de valores o series, en este caso necesitamos por un lado la lista de valores de *Year* y por otro lado la lista de valores de *Month*

In [62]:
year_series = df['Year']
year_series

130870    2012
75478     2012
197183    2012
130772    2012
75479     2012
          ... 
995624    2016
884576    2016
995585    2016
897973    2016
999914    2016
Name: Year, Length: 1036643, dtype: int64

In [63]:
month_series = df['Month']
month_series

130870     1
75478      1
197183     1
130772     1
75479      1
          ..
995624    12
884576    12
995585    12
897973    12
999914    12
Name: Month, Length: 1036643, dtype: int64

Llamamos ahora al objeto *CrossTab* de Pandas, y como primer argumento ponemos la serie o array que desamos como indice (*index*), luego la serie en la cual se basan las columnas (*columns*), y si no establecemos una *aggfunc* utiliza *'size'* por defecto

In [64]:
pd.crosstab(index=year_series,
            columns=month_series)

Month,1,2,3,4,5,6,7,8,9,10,11,12
Year,,,,,,,,,,,,
2012,15614,18123,18604,16590,17014,16911,17718,16492,15613,18515,17416,15025
2013,16638,17119,17397,17685,17786,16421,19085,16660,18946,24546,18617,17398
2014,18013,18214,19839,18077,16639,17762,18867,16021,18970,19579,17486,16937
2015,18245,18116,19122,17603,15572,18413,19127,15507,16656,17785,17335,16180
2016,14515,16130,17282,15223,14487,16418,15319,15333,15376,16191,17335,15036


Como vemos, la lista de valores que le pasamos como argumento fueron objetos del tipo Series

In [65]:
type(year_series)

pandas.core.series.Series

Si los argumentos son de este tipo, hay que tener en cuenta que al crear la tabla *crosstab* utiliza el *indice* para hacer la union entre ambas series. En cambio si los objetos utilizados para crearla son arrays de valores, se basa en el **orden** de los mismos para realizar la union

In [66]:
year_series.values

array([2012, 2012, 2012, ..., 2016, 2016, 2016])

In [67]:
month_series.values

array([ 1,  1,  1, ..., 12, 12, 12])

In [68]:
type(year_series.values)

numpy.ndarray

In [69]:
pd.crosstab(index=year_series.values,
            columns=month_series.values)

col_0,1,2,3,4,5,6,7,8,9,10,11,12
row_0,,,,,,,,,,,,
2012,15614,18123,18604,16590,17014,16911,17718,16492,15613,18515,17416,15025
2013,16638,17119,17397,17685,17786,16421,19085,16660,18946,24546,18617,17398
2014,18013,18214,19839,18077,16639,17762,18867,16021,18970,19579,17486,16937
2015,18245,18116,19122,17603,15572,18413,19127,15507,16656,17785,17335,16180
2016,14515,16130,17282,15223,14487,16418,15319,15333,15376,16191,17335,15036
